In [ ]:

import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()

Saving pharmacologically_active.csv to pharmacologically_active.csv


In [ ]:
df = pd.read_csv('/content/drugbank vocabulary.csv')
df = df.drop(columns=['Accession Numbers', 'CAS', 'UNII', 'Standard InChI Key' ])
#Split Synonyms wherever | is present and make it a separate row
df = df.assign(Synonyms = df['Synonyms'].str.split('|')).explode('Synonyms').reset_index()
#Take only the first word in common name and synonym
df['Common name'] = df['Common name'].str.split().str[0]
df['Synonyms'] = df['Synonyms'].str.split().str[0]
df = df.rename(columns = {'DrugBank ID' : 'Drug'})
df.head()

,index,Drug,Common name,Synonyms
0,0,DB00001,Lepirudin,Hirudin
1,0,DB00001,Lepirudin,Lepirudin
2,1,DB00002,Cetuximab,Cetuximab
3,1,DB00002,Cetuximab,Cétuximab
4,1,DB00002,Cetuximab,Cetuximabum


In [ ]:
df1 = pd.read_csv('/content/pharmacologically_active.csv')
df1 = df1.drop(columns=['ID', 'Name', 'GenBank Protein ID', 'GenBank Gene ID', 'UniProt ID', 'Uniprot Title', 'PDB ID', 'GeneCard ID','GenAtlas ID', 'HGNC ID', 'Species'])
df1 = df1.rename(columns = {'Drug IDs' : 'Drug'})
df1 = df1.assign(Drug = df1['Drug'].str.split(';')).explode('Drug').reset_index()
#Removing spaces from the front as some of the entries have it
df1['Drug'] = df1['Drug'].str.lstrip()
df1

,index,Gene Name,Drug
0,0,ftsI,DB00303
1,1,F13A1,DB11300
2,1,F13A1,DB11311
3,1,F13A1,DB11571
4,1,F13A1,DB13151
...,...,...,...
5837,1312,IGHG1,DB15258
5838,1313,IGHG2,DB15258
5839,1314,IGHG4,DB15258
5840,1315,IGHG3,DB15258


In [ ]:
df2 = pd.merge(df, df1, on = 'Drug', how = 'right')
df2

,index_x,Drug,Common name,Synonyms,index_y,Gene Name
0,292,DB00303,Ertapenem,"(1R,5S,6S,8R,2'S,4'S)-2-(2-(3-carboxyphenylcar...",0,ftsI
1,292,DB00303,Ertapenem,"(4R,5S,6S)-3-((3S,5S)-5-((3-carboxyphenyl)carb...",0,ftsI
2,292,DB00303,Ertapenem,Ertapenem,0,ftsI
3,292,DB00303,Ertapenem,"(1R,5S,6S,8R,2'S,4'S)-2-(2-(3-carboxyphenylcar...",56,mrdA
4,292,DB00303,Ertapenem,"(4R,5S,6S)-3-((3S,5S)-5-((3-carboxyphenyl)carb...",56,mrdA
...,...,...,...,...,...,...
30316,13167,DB15258,Imlifidase,Imlifidase,1314,IGHG4
30317,13167,DB15258,Imlifidase,HMED-IdeS,1315,IGHG3
30318,13167,DB15258,Imlifidase,IdeS,1315,IGHG3
30319,13167,DB15258,Imlifidase,IdeS,1315,IGHG3


In [ ]:
#Getting the verified drug-gene pairs from common name and synonyms
DbPairs = []
for i in range(df2.shape[0]):
  drug = df2['Common name'].iloc[i]
  gene = df2['Gene Name'].iloc[i]
  DrugGene = '('+str(drug).lower()+','+str(gene).lower()+')'
  DbPairs.append(DrugGene)
for i in range(df2.shape[0]):
  drug = df2['Synonyms'].iloc[i]
  gene = df2['Gene Name'].iloc[i]
  DrugGene = '('+str(drug).lower()+','+str(gene).lower()+')'
  DbPairs.append(DrugGene)
DbPairs = list(set(DbPairs))
print(len(DbPairs))

27833


In [ ]:
#Importing the co-occurence matrix
df3 = pd.read_csv('/content/cluster.csv')
DgPairs = []
#Getting unverified drug-gene pairs from the matrix
for column in df3.columns:
  DgPairs.append(column)

DgPairs.pop(0)
print((DgPairs))
df3 = df3.rename(columns = {df3.columns[0]: 'Drug Gene'})
df3

['(flavopiridol,nf-kappab)', '(tnf-r2,tnf-r1)', '(il-2,il-5)', '(il-11,il-10)', '(fgf-7,fgf-2)', '(clopidogrel,p-selectin)', '(fgf-7,fgf-1)', '(il-11,il-13)', '(propranolol,beta2)', '(isoflurane,caspase-3)', '(indomethacin,phospholipase)', '(anti-il-2,cd25)', '(zuclopenthixol,cyp2d6)', '(doxycycline,smad7)', '(il-2,il-1)', '(corticosteroids,leukotriene)', '(gliclazide,niddm)', '(tolbutamide,niddm)', '(dipyridamole,pde5)', '(geldanamycin,hsp90)', '(melatonin,bcl-2)', '(carbamates,faah)', '(pge2,pgd2)', '(losartan,urat1)', '(aldosterone,bmp-6)', '(gm-csf,il-13)', '(prostacyclin,cox-1)', '(astemizole,cyp2j2)', '(prostacyclin,cox-2)', '(gm-csf,il-15)', '(amiloride,kallikrein)', '(diphenhydramine,cyp2d6)', '(gm-csf,il-10)', '(valsartan,adiponectin)', '(forskolin,sf-1)', '(gm-csf,il-11)', '(il-11,gm-csf)', '(capecitabine,dlts)', '(pranlukast,cyslt1)', '(cetuximab,egfr)', '(gm-csf,il-18)', '(cysteamine,gastrin)', '(nadh,nad1)', '(nadh,nad3)', '(telmisartan,abca1)', '(abciximab,p-selectin)', '

,Drug Gene,"(flavopiridol,nf-kappab)","(tnf-r2,tnf-r1)","(il-2,il-5)","(il-11,il-10)","(fgf-7,fgf-2)","(clopidogrel,p-selectin)","(fgf-7,fgf-1)","(il-11,il-13)","(propranolol,beta2)","(isoflurane,caspase-3)","(indomethacin,phospholipase)","(anti-il-2,cd25)","(zuclopenthixol,cyp2d6)","(doxycycline,smad7)","(il-2,il-1)","(corticosteroids,leukotriene)","(gliclazide,niddm)","(tolbutamide,niddm)","(dipyridamole,pde5)","(geldanamycin,hsp90)","(melatonin,bcl-2)","(carbamates,faah)","(pge2,pgd2)","(losartan,urat1)","(aldosterone,bmp-6)","(gm-csf,il-13)","(prostacyclin,cox-1)","(astemizole,cyp2j2)","(prostacyclin,cox-2)","(gm-csf,il-15)","(amiloride,kallikrein)","(diphenhydramine,cyp2d6)","(gm-csf,il-10)","(valsartan,adiponectin)","(forskolin,sf-1)","(gm-csf,il-11)","(il-11,gm-csf)","(capecitabine,dlts)","(pranlukast,cyslt1)",...,"(zanamivir,neuraminidase)","(pravastatin,adiponectin)","(verapamil,calmodulin)","(enalapril,enos)","(abatacept,ctla-4)","(indomethacin,ppargamma)","(daunorubicin,prolidase)","(rosiglitazone,glut4)","(fluoxetine,cyp2d6)","(galanthamine,cholinesterase)","(atorvastatin,fasl)","(atorvastatin,mcm6)","(fibrates,pai-1)","(ritonavir,cd36)","(mitomycin,rad51)","(melatonin,sirt1)","(zidovudine,ifn-alpha)","(losartan,at1r)","(il-2,perlecan)","(ciclosporin,p-gp)","(atra,p-gp)","(disulfiram,5-lipoxygenase)","(verapamil,p-gp)","(cerivastatin,statin)","(carbachol,rgs2)","(doxycycline,vegf)","(collagenase,mmp1)","(ndga,15-lox)","(doxorubicin,mdr1)","(pge2,aromatase)","(erythromycin,p-gp)","(methoxsalen,cyp2a6)","(atra,nf-kappab)","(delavirdine,cyp3a)","(pge2,ifn-gamma)","(dexamethasone,nfkappab)","(propafenone,cyp1a2)","(cisplatin,brca1)","(ndga,nrf2)","(zolmitriptan,5-ht1b)"
0,"(flavopiridol,nf-kappab)",1000,17,6,3,6,10,8,11,15,258,8,39,3,14,5,9,8,13,5,0,18,22,6,10,12,11,14,3,27,4,10,8,17,11,18,4,9,13,13,...,3,9,3,17,20,379,32,9,4,1,16,10,14,13,81,102,6,16,14,9,13,15,4,12,15,26,35,11,1,72,9,2,25,3,8,3,3,9,105,170
1,"(tnf-r2,tnf-r1)",17,1000,290,185,166,13,167,138,34,50,10,100,5,21,285,18,40,39,7,8,105,6,242,3,14,198,58,4,134,175,10,8,215,50,103,132,217,20,29,...,5,17,5,42,55,12,16,88,4,2,43,7,11,6,51,8,119,38,24,8,5,9,8,53,30,154,73,5,20,106,9,5,29,1,183,5,1,29,48,54
2,"(il-2,il-5)",6,290,1000,333,206,11,207,188,9,79,15,19,7,17,407,7,20,46,0,9,141,13,387,1,14,371,30,1,77,283,11,8,410,22,106,192,181,32,12,...,2,22,1,17,43,17,17,123,4,0,55,6,31,16,78,11,120,8,7,17,3,4,11,34,40,99,16,2,22,160,16,1,63,2,274,0,2,56,64,21
3,"(il-11,il-10)",3,185,333,1000,497,9,425,257,4,18,4,29,1,13,355,3,57,24,0,5,60,4,447,1,6,555,23,0,48,537,5,7,340,25,50,431,272,9,5,...,0,20,0,21,39,11,8,61,2,0,26,7,18,7,20,6,319,3,14,5,4,1,3,30,14,80,17,0,5,59,3,0,21,0,243,0,0,21,31,7
4,"(fgf-7,fgf-2)",6,166,206,497,1000,9,488,214,6,8,0,28,0,19,301,4,64,13,0,2,34,2,355,1,4,469,14,1,31,566,2,1,271,24,43,492,300,9,5,...,0,25,1,10,33,10,5,32,0,1,25,5,10,10,9,7,562,4,14,2,10,0,1,28,13,77,17,1,3,22,0,0,11,0,255,2,0,13,13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3509,"(dexamethasone,nfkappab)",3,5,0,0,2,0,1,1,17,0,74,0,11,7,1,23,4,6,416,100,1,72,0,173,21,0,13,746,15,0,106,41,0,3,0,2,0,6,19,...,236,5,697,14,5,13,43,2,182,586,1,70,4,4,4,18,3,29,15,31,9,126,109,74,5,6,0,297,2,0,28,321,4,313,0,1000,313,83,0,2
3510,"(propafenone,cyp1a2)",3,1,2,0,0,1,3,1,10,7,54,3,179,9,0,10,7,11,261,67,1,92,1,188,15,0,18,387,8,0,97,51,1,3,1,2,0,11,10,...,89,8,316,18,12,12,54,3,291,226,3,65,6,5,8,12,1,16,23,97,14,146,132,52,11,26,3,230,52,8,96,273,2,1000,2,313,1000,69,2,2
3511,"(cisplatin,brca1)",9,29,56,21,13,21,12,25,14,18,32,10,59,102,8,13,48,29,46,20,32,20,19,35,34,9,26,77,25,15,37,39,12,21,28,20,15,101,20,...,28,97,97,26,37,35,20,18,36,36,34,48,45,87,53,24,21,7,27,61,62,23,57,43,100,106,14,65,58,14,

In [ ]:
import random
#Drug-gene pairs that are in both the co-occurence matrix and drugbank
common = []
for pair in DgPairs:
  if pair in DbPairs:
    common.append(pair)
print(len(common))
uncommon = list(set(DgPairs) - set(common))
#Making the seed-set with 10 random drug-gene pairs that are in both the co-occurence matrix and drugbank
seed_set = random.sample(common, )
#taking pairs that are not in seed-set as test is supposed to be mutually exclusive with it
pairs = list(set(common) - set(seed_set))
#taking 50 drug-gene pairs that are common to both drugbank and our matrix
test_set1 = random.sample(pairs, 50)
#taking 50 pairs that are only in our matrix
test_set2 = random.sample(uncommon, 50)
test_set = test_set1 + test_set2
print(seed_set)
print(test_set)

67
['(abx-egf,egfr)', '(dasatinib,epha2)', '(hyaluronate,cd44)', '(melatonin,mtnr1b)', '(plerixafor,cxcr4)', '(orlistat,fasn)', '(lapatinib,egfr)', '(haloperidol,drd2)', '(dantrolene,ryr1)', '(sildenafil,pde5a)']
['(imiquimod,tlr7)', '(pgi2,ptgis)', '(warfarin,vkorc1)', '(gefitinib,egfr)', '(ezetimibe,npc1l1)', '(g-csf,csf3r)', '(doxorubicin,top2a)', '(aripiprazole,drd2)', '(vasopressin,avpr1b)', '(cerulenin,fasn)', '(creatine,slc6a8)', '(sorafenib,braf)', '(proguanil,dhfr)', '(naltrexone,oprm1)', '(vemurafenib,braf)', '(capsaicin,trpv1)', '(panitumumab,egfr)', '(pyridoxal,pdxk)', '(rapamycin,mtor)', '(thyrotropin,tshr)', '(gemcitabine,rrm1)', '(metyrapone,cyp11b1)', '(lapatinib,erbb2)', '(probucol,abca1)', '(erlotinib,egfr)', '(ciprofloxacin,parc)', '(entacapone,comt)', '(sorafenib,flt3)', '(sirolimus,mtor)', '(methotrexate,dhfr)', '(cetuximab,egfr)', '(temsirolimus,mtor)', '(vildagliptin,dpp4)', '(torcetrapib,cetp)', '(everolimus,mtor)', '(sitagliptin,dpp4)', '(etoposide,top2a)', '(m

In [ ]:
#Scoring function
scores = {}
for t in test_set:
  val = []
  ranks = {}
  #if the pair in test_set is in co-occurence matrix
  if t in DgPairs:
    #then sort the df on the basis of the column corresponding to the pair
    sorted_mat = df3.sort_values(by = t, ascending = False)
    rank = 1
    #Assigning ranks to all the drug-gene pair on the basis of their position 
    for drugGene in sorted_mat['Drug Gene']:
      ranks[drugGene] = rank
      rank += 1
    #Taking the ranksum that correspond to the seed_set members 
    for DG in sorted_mat['Drug Gene']:
      if DG in seed_set:
        val.append(ranks[DG])
        #The ranksum are the scores
    scores[t] = sum(val)

print(scores)
sorted_mat


{'(imiquimod,tlr7)': 12859, '(pgi2,ptgis)': 20853, '(warfarin,vkorc1)': 20322, '(gefitinib,egfr)': 7026, '(ezetimibe,npc1l1)': 7901, '(g-csf,csf3r)': 8330, '(doxorubicin,top2a)': 10576, '(aripiprazole,drd2)': 13594, '(vasopressin,avpr1b)': 16728, '(cerulenin,fasn)': 14211, '(creatine,slc6a8)': 18920, '(sorafenib,braf)': 13345, '(proguanil,dhfr)': 11484, '(naltrexone,oprm1)': 8917, '(vemurafenib,braf)': 9796, '(capsaicin,trpv1)': 14950, '(panitumumab,egfr)': 6324, '(pyridoxal,pdxk)': 11180, '(rapamycin,mtor)': 8749, '(thyrotropin,tshr)': 15565, '(gemcitabine,rrm1)': 11338, '(metyrapone,cyp11b1)': 11876, '(lapatinib,erbb2)': 12986, '(probucol,abca1)': 9744, '(erlotinib,egfr)': 5532, '(ciprofloxacin,parc)': 11338, '(entacapone,comt)': 11802, '(sorafenib,flt3)': 15156, '(sirolimus,mtor)': 11142, '(methotrexate,dhfr)': 6772, '(cetuximab,egfr)': 7261, '(temsirolimus,mtor)': 11496, '(vildagliptin,dpp4)': 14200, '(torcetrapib,cetp)': 9937, '(everolimus,mtor)': 10529, '(sitagliptin,dpp4)': 1373

,Drug Gene,"(flavopiridol,nf-kappab)","(tnf-r2,tnf-r1)","(il-2,il-5)","(il-11,il-10)","(fgf-7,fgf-2)","(clopidogrel,p-selectin)","(fgf-7,fgf-1)","(il-11,il-13)","(propranolol,beta2)","(isoflurane,caspase-3)","(indomethacin,phospholipase)","(anti-il-2,cd25)","(zuclopenthixol,cyp2d6)","(doxycycline,smad7)","(il-2,il-1)","(corticosteroids,leukotriene)","(gliclazide,niddm)","(tolbutamide,niddm)","(dipyridamole,pde5)","(geldanamycin,hsp90)","(melatonin,bcl-2)","(carbamates,faah)","(pge2,pgd2)","(losartan,urat1)","(aldosterone,bmp-6)","(gm-csf,il-13)","(prostacyclin,cox-1)","(astemizole,cyp2j2)","(prostacyclin,cox-2)","(gm-csf,il-15)","(amiloride,kallikrein)","(diphenhydramine,cyp2d6)","(gm-csf,il-10)","(valsartan,adiponectin)","(forskolin,sf-1)","(gm-csf,il-11)","(il-11,gm-csf)","(capecitabine,dlts)","(pranlukast,cyslt1)",...,"(zanamivir,neuraminidase)","(pravastatin,adiponectin)","(verapamil,calmodulin)","(enalapril,enos)","(abatacept,ctla-4)","(indomethacin,ppargamma)","(daunorubicin,prolidase)","(rosiglitazone,glut4)","(fluoxetine,cyp2d6)","(galanthamine,cholinesterase)","(atorvastatin,fasl)","(atorvastatin,mcm6)","(fibrates,pai-1)","(ritonavir,cd36)","(mitomycin,rad51)","(melatonin,sirt1)","(zidovudine,ifn-alpha)","(losartan,at1r)","(il-2,perlecan)","(ciclosporin,p-gp)","(atra,p-gp)","(disulfiram,5-lipoxygenase)","(verapamil,p-gp)","(cerivastatin,statin)","(carbachol,rgs2)","(doxycycline,vegf)","(collagenase,mmp1)","(ndga,15-lox)","(doxorubicin,mdr1)","(pge2,aromatase)","(erythromycin,p-gp)","(methoxsalen,cyp2a6)","(atra,nf-kappab)","(delavirdine,cyp3a)","(pge2,ifn-gamma)","(dexamethasone,nfkappab)","(propafenone,cyp1a2)","(cisplatin,brca1)","(ndga,nrf2)","(zolmitriptan,5-ht1b)"
3153,"(chloramphenicol,pai-1)",23,21,37,10,9,11,13,12,6,32,7,12,18,458,34,1,152,25,1,2,28,9,30,10,55,21,83,6,19,6,3,7,45,23,33,20,26,519,9,...,3,472,4,16,28,160,20,37,0,3,22,13,13,373,272,20,49,3,15,22,279,5,5,7,511,36,45,6,24,49,14,3,190,6,37,6,6,82,27,60
1211,"(chloramphenicol,pomc)",23,21,37,10,9,10,13,13,6,32,7,13,18,461,34,1,148,25,1,2,28,9,30,10,54,21,82,7,19,6,3,7,45,22,33,20,26,515,9,...,3,474,4,18,27,159,19,37,0,3,22,13,13,371,271,19,49,3,15,21,272,5,5,7,529,37,46,6,24,49,14,3,188,6,37,7,6,83,27,59
1163,"(chloramphenicol,beta-actin)",22,28,42,11,9,9,12,12,5,34,8,11,18,461,37,2,148,26,2,3,31,9,32,10,54,24,74,6,20,7,4,7,51,22,32,20,28,526,8,...,4,471,5,18,25,159,20,41,0,4,22,13,15,379,271,21,52,3,15,23,277,5,6,7,518,37,45,5,23,55,15,3,193,6,43,7,6,77,27,62
2870,"(chloramphenicol,mdr1)",22,28,42,11,9,10,12,12,5,34,8,11,18,466,37,1,149,26,2,3,31,9,32,10,51,24,74,7,20,7,4,7,51,21,32,21,27,528,8,...,4,464,5,18,25,157,20,41,0,4,22,13,15,372,269,22,52,3,15,23,273,5,6,7,519,37,45,5,24,55,15,3,190,6,43,8,6,81,27,62
1538,"(chloramphenicol,ifn-beta)",25,33,49,14,8,10,11,13,5,37,9,12,18,421,38,1,130,26,1,3,31,11,34,12,50,25,80,6,21,8,4,9,54,21,36,18,28,474,8,...,3,437,4,18,26,142,19,44,0,3,23,13,15,344,247,22,48,3,16,19,244,6,5,8,478,40,46,4,26,59,16,2,175,6,44,7,6,79,31,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1570,"(vemurafenib,braf)",1,9,2,0,1,2,0,1,29,2,103,2,6,0,2,25,2,44,242,244,2,79,1,74,7,1,14,312,57,1,132,59,3,1,3,2,2,0,30,...,351,0,301,8,3,8,39,8,227,311,1,46,4,1,9,6,0,74,19,10,0,123,197,113,0,2,3,176,2,3,14,263,1,167,1,289,167,31,0,4
3482,"(fluoxetine,cyp2d6)",4,4,4,2,0,3,1,1,15,8,168,2,87,0,1,17,3,9,220,165,0,126,1,100,18,1,6,204,22,0,113,98,2,4,3,0,1,0,20,...,203,1,203,13,4,7,71,11,1000,196,4,42,2,2,1,11,3,35,10,52,3,162,171,62,0,18,2,130,45,9,64,304,6,291,0,182,291,36,2,2
434,"(ranibizumab,vegf-a)",11,6,1,0,0,7,1,4,431,14,106,12,7,1,31,462,4,4,70,63,0,29,0,44,6,1,9,105,30,3,39,22,1,0,1,23,7,0,406,...,121,0,82,6,87,11,57,5,50,146,5,18,0,0,1,17,5,471,69,5,2,66,

In [ ]:
#scores
dg = list(scores.keys())
sc = list(scores.values())
di = {'Drug, Gene': dg, 'Scores': sc, 'size': len(seed_set)}
df4 = pd.DataFrame(di)
df4.head(10)

,"Drug, Gene",Scores,size
0,"(imiquimod,tlr7)",12859,10
1,"(pgi2,ptgis)",20853,10
2,"(warfarin,vkorc1)",20322,10
3,"(gefitinib,egfr)",7026,10
4,"(ezetimibe,npc1l1)",7901,10
5,"(g-csf,csf3r)",8330,10
6,"(doxorubicin,top2a)",10576,10
7,"(aripiprazole,drd2)",13594,10
8,"(vasopressin,avpr1b)",16728,10
9,"(cerulenin,fasn)",14211,10
